In [71]:
import csv
import json
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import time
from pprint import pprint
from datetime import datetime
import pytz
import shutil

In [72]:
version='23_24_gw2'
league_code = "551115"

In [73]:
start=time.perf_counter()

In [74]:
#Request league standings and return all league data 
def get_json_from_url(league_code):
    league_url="https://fantasy.premierleague.com/api/leagues-classic/"+league_code+"/standings/?page_new_entries=1&page_standings=1&phase=1"
    get=requests.get(league_url)
    league_data=json.loads(get.text)
    league_name = league_data['league'].get('name')
    print("Data fetched, league name: ", league_name)
    return league_data, league_name

#Use league data to return a dict of names & entry ids
def make_ids(league_data):
    member_ids={}
    for member in league_data['standings']['results']:
        name=member.get('player_name')
        mid=member.get('entry')
        member_ids[name]=mid
    return member_ids

#Use first value in member_ids to find the max gameweek that data is available for
def find_max_gw(xid): #xid=member entry id
    gw=1
    while gw < 100:
        try:
            get_mgw=requests.get("https://fantasy.premierleague.com/api/entry/"+str(xid)+"/event/"+str(gw)+"/picks/")
            data_mgw=json.loads(get_mgw.text)
            list(data_mgw['entry_history'].keys())
            gw+=1
        except KeyError as err:
            print("Max gameweek data available for is: ",gw)
            return (gw)
            break

In [75]:
#(Doesn't work until GW1)
league_data, league_name = get_json_from_url(league_code)
member_ids=make_ids(league_data)
mgw=find_max_gw(list(member_ids.values())[0])

Data fetched, league name:  RIVERSIDE LEAGUE X 2023/24
Max gameweek data available for is:  3


In [76]:
#Get data1
def get_first_member_data(member_ids):
    xid=list(member_ids.values())[0]
    gw=1
    get_first=requests.get("https://fantasy.premierleague.com/api/entry/"+str(xid)+"/event/"+str(gw)+"/picks/")
    data_first=json.loads(get_first.text)
    return data_first

#Member picks
def dataframe_from_ids(member_ids,cols):
    p=[]

     #member data

    for xid in member_ids.values():
        for gw in range(1,mgw):    
            get_gw=requests.get("https://fantasy.premierleague.com/api/entry/"+str(xid)+"/event/"+str(gw)+"/picks/")
            data_gw=json.loads(get_gw.text)
            try:
                for x in range(0,15):
                    #picks
                    # print("Requesting...... ",xid,"GW "+str(gw),"Pick "+str(x+1))
                    row=data_gw["picks"][x]

                    name=list(member_ids.keys())[list(member_ids.values()).index(xid)]
                    selected_row = [name,xid,gw,x+1]

                    for i in cols:
                        selected_row.append(row.get(i))

                    p.append(selected_row)

                    print(xid,name,"GW "+str(gw),"Pick "+str(x+1))
            except KeyError as e:
                name = list(member_ids.keys())[list(member_ids.values()).index(xid)]
                print("GW Could not be found!",xid,name,"GW "+str(gw))
                print(e)
                continue
    print("Finished")
    pcols=['member','memberid','gw','pick']
    pcols.extend(cols)
    picksdf = pd.DataFrame(p, columns=pcols)
    return picksdf
data_for_columns=get_first_member_data(member_ids)
cols=list(data_for_columns["picks"][0].keys())
picksdf=dataframe_from_ids(member_ids,cols)

774689 Ryan Boyce GW 1 Pick 1
774689 Ryan Boyce GW 1 Pick 2
774689 Ryan Boyce GW 1 Pick 3
774689 Ryan Boyce GW 1 Pick 4
774689 Ryan Boyce GW 1 Pick 5
774689 Ryan Boyce GW 1 Pick 6
774689 Ryan Boyce GW 1 Pick 7
774689 Ryan Boyce GW 1 Pick 8
774689 Ryan Boyce GW 1 Pick 9
774689 Ryan Boyce GW 1 Pick 10
774689 Ryan Boyce GW 1 Pick 11
774689 Ryan Boyce GW 1 Pick 12
774689 Ryan Boyce GW 1 Pick 13
774689 Ryan Boyce GW 1 Pick 14
774689 Ryan Boyce GW 1 Pick 15
774689 Ryan Boyce GW 2 Pick 1
774689 Ryan Boyce GW 2 Pick 2
774689 Ryan Boyce GW 2 Pick 3
774689 Ryan Boyce GW 2 Pick 4
774689 Ryan Boyce GW 2 Pick 5
774689 Ryan Boyce GW 2 Pick 6
774689 Ryan Boyce GW 2 Pick 7
774689 Ryan Boyce GW 2 Pick 8
774689 Ryan Boyce GW 2 Pick 9
774689 Ryan Boyce GW 2 Pick 10
774689 Ryan Boyce GW 2 Pick 11
774689 Ryan Boyce GW 2 Pick 12
774689 Ryan Boyce GW 2 Pick 13
774689 Ryan Boyce GW 2 Pick 14
774689 Ryan Boyce GW 2 Pick 15
1224690 Stephen Collier GW 1 Pick 1
1224690 Stephen Collier GW 1 Pick 2
1224690 Stephen 

In [77]:
#Insert into PSQL
from sqlalchemy import create_engine, text
from datetime import datetime
import psycopg2 # python -> psql connection
import psycopg2.extras

#Import secret settings
import CJDH_local_settings
dbname = CJDH_local_settings.local_settings['TRDL_PSQL_Analysis']['dbname']
user = CJDH_local_settings.local_settings['TRDL_PSQL_Analysis']['user']
password = CJDH_local_settings.local_settings['TRDL_PSQL_Analysis']['password']
host = "localhost"
port = 5432

#Create connection to psql via psycopg2 & sqlalchemy
connection_string = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}'
engine = create_engine(connection_string)
engine.autocommit = True  # read documentation understanding when to Use & NOT use # changes will be automatically committed to the database after each statement is executed.

In [78]:
picksdf.columns
picks = picksdf[['member', 'memberid', 'gw', 'pick', 'element', 'position', 'multiplier',
       'is_captain', 'is_vice_captain']]

In [79]:
#Create staging table for player data 
def create_staging_table_picks(cursor):
    cursor.execute(text("""
        DROP TABLE IF EXISTS picks CASCADE;
        CREATE UNLOGGED TABLE picks (
            member          TEXT,
            memberid        NUMERIC,
            gw              NUMERIC,
            pick            NUMERIC,
            element         NUMERIC,
            position        NUMERIC,
            multiplier      NUMERIC,
            is_captain      BOOLEAN,
            is_vice_captain BOOLEAN
        );"""))
# creating our schema  and sending the table to psql
with engine.connect() as cursor:
    create_staging_table_picks(cursor)
picks.to_sql('picks', con=engine, if_exists='replace',index=False)

555

In [91]:
#Query gets only player details from the latest GW
query = """
        ALTER TABLE player_details
            RENAME COLUMN total_points TO total_points_gw;

        WITH playergw AS (
            SELECT *,
            ROW_NUMBER() OVER(
                    PARTITION BY
                        season,
                        code,
                        event,
                        fixture_id --Only one record per GW
                    ORDER BY 
                        now_datetime_utc DESC) as rn --Only select the latest date
        FROM playergw
        ),
        currentgw AS (
            SELECT MAX(current_gw) as max_gw
            FROM playergw
        )
        
        SELECT *
        FROM player_details
        LEFT JOIN playergw
            ON player_details.element = playergw.id
            AND player_details.round = playergw.current_gw
        CROSS JOIN currentgw
        WHERE player_details.round = currentgw.max_gw
            AND playergw.rn = 1
        ;
        """
with engine.connect() as conn:
    playerdf = pd.read_sql_query(query, conn)
print(playerdf.shape)

pd.pivot_table(playerdf, index=['id','web_name','team_name','round'],values='total_points_gw',aggfunc=np.sum).sort_values(by='total_points_gw', ascending=False)

(595, 133)


,,,,total_points_gw
id,web_name,team_name,round,
108,Mbeumo,Brentford,2,16
140,March,Brighton,2,15
34,Bailey,Aston Villa,2,14
294,Diogo J.,Liverpool,2,12
143,Mitoma,Brighton,2,12
...,...,...,...,...
551,Dawson,Wolves,2,-1
391,Martinez,Man Utd,2,-1
566,Matheus N.,Wolves,2,-1


In [81]:
max_gw = max(picksdf['gw'])
me = 'Christopher Harris'
myid = set(picksdf[picksdf['member']==me]['memberid'])
mypicksdf = picksdf[(picksdf['member']==me)&(picksdf['gw']==max_gw)]
mypicks = set(mypicksdf['element'])
mypicks

#league_standings
l = pd.DataFrame(league_data['standings']['results'])
mytotal = l[l['entry'].isin(myid)]['total'].values
#get ids for members above me or less than 100 points below me
ids_close_to_me = set(l[l['total']<=int(mytotal)+100]['entry'][~l['entry'].isin(myid)])

In [82]:
#Get ep_next * by ownership rate in mini-league
latestpicks = picksdf[(picksdf['gw']==max(picksdf['gw']))&(picksdf['memberid'].isin(ids_close_to_me))]
max_owns = len(set(latestpicks['memberid']))

ownership = pd.pivot_table(latestpicks, index=['element'],values='multiplier',aggfunc=np.sum).reset_index()
ownership['own_rate'] = ownership['multiplier']/max_owns

ownership.sort_values(by='own_rate', ascending=False)

playerdf = pd.merge(playerdf, ownership, how='outer',on='element')
filtered = playerdf['element'].isin(mypicks)
gainpicks = playerdf[filtered].merge(mypicksdf,how='left',on='element',suffixes=('','_myteam'))

gainpicks['expected_gain'] = gainpicks['ep_next'].astype(float)*(gainpicks['multiplier_myteam']-gainpicks['own_rate'])
gainpicks[['web_name','element','element_type','multiplier','multiplier_myteam','own_rate','ep_next','expected_gain']].sort_values(by='expected_gain',ascending=False)

,web_name,element,element_type,multiplier,multiplier_myteam,own_rate,ep_next,expected_gain
8,Haaland,355,4.0,27.0,2,1.500000,9.0,4.500000
13,Onana,597,1.0,6.0,1,0.333333,6.0,4.000000
3,Watkins,60,4.0,5.0,1,0.277778,5.5,3.972222
4,Estupiñan,131,2.0,11.0,1,0.611111,9.0,3.500000
6,Chilwell,195,2.0,5.0,1,0.277778,4.5,3.250000
5,Mitoma,143,3.0,12.0,1,0.666667,8.5,2.833333
9,B.Fernandes,373,3.0,4.0,1,0.222222,2.5,1.944444
0,Martinelli,12,3.0,7.0,1,0.388889,3.0,1.833333
1,Saka,19,3.0,13.0,1,0.722222,5.5,1.527778
10,Rashford,396,3.0,9.0,1,0.500000,3.0,1.500000


In [83]:
print("Expected Total Gain on Division Average")
gainpicks['expected_gain'].sum()

Expected Total Gain on Division Average


28.63333333333333

In [84]:
#players not in my picks
losspicks = playerdf[~filtered].copy().dropna(subset='multiplier')
losspicks['expected_loss'] = losspicks['ep_next'].astype(float)*(0-losspicks['own_rate'])
print("Expected Total Loss on Division Average: ")
print(losspicks['expected_loss'].sum())
losspicks[['web_name','element','multiplier','own_rate','ep_next','expected_loss']].sort_values(by='expected_loss',ascending=True).head()

Expected Total Loss on Division Average: 
-21.344444444444445


,web_name,element,multiplier,own_rate,ep_next,expected_loss
277,Salah,308,8.0,0.444444,5.0,-2.222222
133,João Pedro,135,10.0,0.555556,3.5,-1.944444
106,Mbeumo,108,3.0,0.166667,11.5,-1.916667
263,Diogo J.,294,3.0,0.166667,6.5,-1.083333
340,Shaw,398,5.0,0.277778,3.5,-0.972222


In [85]:
print("Net Gain")
print(round(gainpicks['expected_gain'].sum() + losspicks['expected_loss'].sum(),2))

Net Gain
7.29


In [86]:
teamgains = gainpicks[['web_name','status','element','element_type','now_cost','multiplier','multiplier_myteam','own_rate','ep_next','expected_gain']].sort_values(by='expected_gain',ascending=False)
teamgains['expected_loss_on_transfer'] = teamgains['ep_next'].astype(float)*(teamgains['own_rate'])

allgains = playerdf.copy()
allgains['expected_gain'] = allgains['ep_next'].astype(float)*(1-allgains['own_rate'])

def top_suggested_transfer(row, df):
    #get element, element_type & gain
    myteam_type = row['element_type']
    # myteam_gain = row['expected_gain']
    myteam_loss = row['expected_loss_on_transfer']
    myteam_cost = row['now_cost']
    myteam_element = row['element']
    #get playerdf element, element_type & gain
    filtered = (df['element_type']==myteam_type) & (df['expected_gain']>myteam_loss) & (df['now_cost']<=myteam_cost) & (df['element']!=myteam_element)
    best_gain = df[filtered].sort_values(by='expected_gain', ascending=False).head(1)
    best_gain['own_rate'] = round(best_gain['own_rate'],3)
    best_gain['expected_gain'] = round(best_gain['expected_gain'],2)
    best_gain['diff'] = round(df['expected_gain']-row['expected_loss_on_transfer'],2)
    return best_gain[['web_name','status','own_rate','ep_next','expected_gain','diff']].values[0]

teamgains['affordable_suggested_transfer'] = teamgains.apply(lambda x: top_suggested_transfer(x, allgains),axis=1)
# Extracting the second element from the arrays in the "affordable_suggested_transfer" column
teamgains['suggested_transfer_gain'] = teamgains['affordable_suggested_transfer'].apply(lambda x: x[-1])

# Sorting the DataFrame based on the second element in the arrays
teamgains = teamgains.sort_values(by='suggested_transfer_gain', ascending=False)
teamgains.head(20)

,web_name,status,element,element_type,now_cost,multiplier,multiplier_myteam,own_rate,ep_next,expected_gain,expected_loss_on_transfer,affordable_suggested_transfer,suggested_transfer_gain
9,B.Fernandes,a,373,3.0,85.0,4.0,1,0.222222,2.5,1.944444,0.555556,"[March, a, 0.056, 12.0, 11.33, 10.78]",10.78
0,Martinelli,a,12,3.0,80.0,7.0,1,0.388889,3.0,1.833333,1.166667,"[March, a, 0.056, 12.0, 11.33, 10.17]",10.17
10,Rashford,a,396,3.0,90.0,9.0,1,0.500000,3.0,1.500000,1.500000,"[March, a, 0.056, 12.0, 11.33, 9.83]",9.83
1,Saka,a,19,3.0,86.0,13.0,1,0.722222,5.5,1.527778,3.972222,"[March, a, 0.056, 12.0, 11.33, 7.36]",7.36
7,James,i,206,2.0,55.0,0.0,1,0.000000,0.0,0.000000,0.000000,"[Walker, a, 0.056, 7.5, 7.08, 7.08]",7.08
2,Turner,a,28,1.0,40.0,3.0,0,0.166667,1.5,-0.250000,0.250000,"[Areola, a, 0.0, 6.5, 6.5, 6.25]",6.25
6,Chilwell,a,195,2.0,57.0,5.0,1,0.277778,4.5,3.250000,1.250000,"[Walker, a, 0.056, 7.5, 7.08, 5.83]",5.83
5,Mitoma,a,143,3.0,65.0,12.0,1,0.666667,8.5,2.833333,5.666667,"[March, a, 0.056, 12.0, 11.33, 5.67]",5.67
3,Watkins,a,60,4.0,80.0,5.0,1,0.277778,5.5,3.972222,1.527778,"[J.Alvarez, a, 0.056, 7.5, 7.08, 5.56]",5.56
13,Onana,a,597,1.0,50.0,6.0,1,0.333333,6.0,4.000000,2.000000,"[Leno, a, 0.0, 6.5, 6.5, 4.5]",4.50


In [93]:
print("Sorted by Expected Gain")
allgains[['web_name','status','element','element_type','now_cost','ep_next','own_rate','expected_gain']].sort_values(by='expected_gain',ascending=False).dropna().head(20)

Sorted by Expected Gain


,web_name,status,element,element_type,now_cost,ep_next,own_rate,expected_gain
138,March,a,140,3.0,65.0,12.0,0.055556,11.333333
106,Mbeumo,a,108,3.0,66.0,11.5,0.166667,9.583333
307,Rodrigo,a,365,3.0,56.0,9.5,0.055556,8.972222
272,Luis Díaz,a,303,3.0,76.0,7.5,0.055556,7.083333
285,J.Alvarez,a,343,4.0,65.0,7.5,0.055556,7.083333
294,Ederson M.,a,352,1.0,55.0,7.5,0.055556,7.083333
311,Walker,a,369,2.0,50.0,7.5,0.055556,7.083333
357,Isak,a,415,4.0,76.0,7.5,0.111111,6.666667
295,Foden,a,353,3.0,75.0,7.5,0.111111,6.666667
343,Wan-Bissaka,a,401,2.0,45.0,7.0,0.055556,6.611111


In [ ]:
finish=time.perf_counter()
print(f'Finished in {round(((finish-start)/60),2)} minute(s)')
#17/08/23: Finished in 0.43 minute(s)

Finished in 0.43 minute(s)
